In [1]:
import pandas as pd
import os
import fnmatch

https://www.understandingsociety.ac.uk/documentation/mainstage/variables/tenure_dv/

https://www.understandingsociety.ac.uk/documentation/mainstage/variables/ff_hhsize/

https://www.understandingsociety.ac.uk/documentation/mainstage/variables/urban_dv/

In [19]:
df_hh_data =pd.read_csv('../data/Survey/n_hhresp.tab', sep='\t')

In [20]:
df_hh_data.head()

,n_hidp,n_intnum,n_hhorig,n_psu,n_strata,n_month,n_quarter,n_ivfho,n_hglieng,n_hglitrans,...,n_nemp_dv,n_nue_dv,n_nwage_dv,n_nchoecd_dv,n_nadoecd_dv,n_ieqmoecd_dv,n_tenure_dv,n_fihhnegsei_if,n_fihhmngrs_if,n_hhdeng2_xw
0,68006826,21008765,1,2012,2006,3,1,11,1,0,...,0,2,2,0,2,1.5,6,0,0.3796,0.959796
1,68013626,-8,1,2012,2006,3,1,10,-8,0,...,1,0,0,0,1,1.0,2,0,0.0000,1.669569
2,68020426,-8,1,2012,2006,3,1,10,-8,0,...,2,0,2,2,2,2.1,1,0,0.0000,0.365966
3,68027226,-8,1,2012,2006,3,1,10,-8,0,...,2,0,2,0,2,1.5,2,0,0.0000,0.782824
4,68047626,-8,1,2060,2030,3,1,10,-8,0,...,1,1,2,3,2,2.4,2,0,0.0000,1.415236


In [21]:
for item in sorted(df_hh_data.columns):
    print(item)


n_addrchck
n_agechy_dv
n_babybirthy1
n_babybirthy2
n_babynum
n_babysex1
n_babysex2
n_carval
n_cdelply
n_cdepdo2
n_cdepdo3
n_cdepdo4
n_cdepdo5
n_cdepdo6
n_cdephave1
n_cdephave2
n_cdephave3
n_cdephave4
n_cdeptrp
n_cdtv
n_cduse1
n_cduse12
n_cduse13
n_cduse2
n_cduse5
n_cduse6
n_cduse7
n_cduse8
n_cduse9
n_cduse96
n_cordet1
n_cordet10
n_cordet11
n_cordet12
n_cordet13
n_cordet14
n_cordet15
n_cordet16
n_cordet17
n_cordet18
n_cordet19
n_cordet2
n_cordet20
n_cordet21
n_cordet22
n_cordet23
n_cordet24
n_cordet25
n_cordet26
n_cordet27
n_cordet28
n_cordet29
n_cordet3
n_cordet30
n_cordet31
n_cordet32
n_cordet4
n_cordet5
n_cordet6
n_cordet7
n_cordet8
n_cordet9
n_cordet96
n_coreswho1
n_coreswho10
n_coreswho11
n_coreswho12
n_coreswho13
n_coreswho14
n_coreswho15
n_coreswho16
n_coreswho2
n_coreswho3
n_coreswho4
n_coreswho5
n_coreswho6
n_coreswho7
n_coreswho8
n_coreswho9
n_coreswho96
n_country
n_cplay
n_ctband_dv
n_ctband_if
n_deskspace
n_duelpay
n_elecpay
n_fborn
n_ff_hsbeds
n_ff_hsownd
n_ff_hsrooms
n_fih

In [22]:
df_hh_data['n_tenure_dv'].unique()

array([ 6,  2,  1,  4,  3,  8, -9,  5])

In [23]:
df_hh_data['n_hhsize'].unique()

array([ 2,  1,  4,  5,  3,  6, 10,  9,  8,  7, 12, 11, 13])

In [24]:
df_hh_data['n_urban_dv'].unique()

array([ 1,  2, -9])